In [1]:
from requests import Session, get
from datetime import datetime
import json
import re
session = Session()

In [2]:
initHeaders=dict()
h='''accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: html
accept-language: ko
sec-ch-ua: "Microsoft Edge";v="107", "Chromium";v="107", "Not=A?Brand";v="24"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: none
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'''

for line in h.split('\n'):
    res = line.split(': ')
    initHeaders[res[0]]=res[1]

initURL = 'http://m.facebook.com/'
init_resp = session.get(initURL, headers=initHeaders)
'''
initHTML = init_resp.text
start = initHTML.index('nc_x=')
end = initHTML.index('" ',start)
nc_x = initHTML[start+5:end]
'''

'\ninitHTML = init_resp.text\nstart = initHTML.index(\'nc_x=\')\nend = initHTML.index(\'" \',start)\nnc_x = initHTML[start+5:end]\n'

In [3]:
'''
init_resp.headers['Set-Cookie']

setCookieReq = init_resp.headers['Set-Cookie'].split('SameSite=None')[:-1]

for cookieKV  in setCookieReq:
    if cookieKV[:2] == ', ':
        cookieKV=cookieKV[2:]
    
    vStart = cookieKV.index('=')
    vEnd = cookieKV.index(';')
    
    print(cookieKV[:vStart])
    print(cookieKV[vStart+1:vEnd])'''

"\ninit_resp.headers['Set-Cookie']\n\nsetCookieReq = init_resp.headers['Set-Cookie'].split('SameSite=None')[:-1]\n\nfor cookieKV  in setCookieReq:\n    if cookieKV[:2] == ', ':\n        cookieKV=cookieKV[2:]\n    \n    vStart = cookieKV.index('=')\n    vEnd = cookieKV.index(';')\n    \n    print(cookieKV[:vStart])\n    print(cookieKV[vStart+1:vEnd])"

In [4]:
session.cookies # 쿠키 확인

<RequestsCookieJar[Cookie(version=0, name='datr', value='BcFgYz5oGxR0lki9Lt5jinXy', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1730357254, discard=False, comment=None, comment_url=None, rest={'httponly': None, 'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='fr', value='0r24tPgwuE3BGa14a..BjYMEF.5I.AAA.0.0.BjYMEF.AWVoqbx-Bsg', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1675061253, discard=False, comment=None, comment_url=None, rest={'httponly': None, 'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='sb', value='BcFgY1TGqeaH7_pXlzry0qTd', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1730357254, discard=False, comment=None, comment

In [5]:
loginHTML = init_resp.text
cropstart='action="/login/device-based/regular/login/?refsrc=deprecated&amp;lwv=100&amp;refid=8"'
cropend="비밀번호를 잊으셨나요?"
paramHTML = loginHTML[loginHTML.index(cropstart):loginHTML.index(cropend)]

In [6]:
startpoint = '<input type="hidden"'
endpoint = ">"
prev_end=0

payloadFromHTML=dict()
for n in range(10):
    try:
        start = paramHTML.index(startpoint,prev_end)
        end = paramHTML.index(endpoint,start)
    except:
        break
    KVstring = paramHTML[start:end]
    
    if "value" in KVstring:
        kv = re.findall('.+name="([^"]+)".+value="([^"]+)"',KVstring)
        payloadFromHTML[kv[0][0]] = kv[0][1]
    else:
        pass
    prev_end=end

In [7]:
payloadFromHTML

{'lsd': 'AVpWTQni-pw',
 'jazoest': '21006',
 'm_ts': '1667285253',
 'li': 'BcFgY8tkaQnkytnPUlKQ4OCB',
 'try_number': '0',
 'unrecognized_tries': '0'}

In [8]:
cTime = payloadFromHTML['m_ts']

In [9]:
payloadFromHTML['email']='seungirumd+1@gmail.com'
payloadFromHTML['encpass'] = '#PWD_BROWSER:0:{}:{}'.format(cTime,'pfbmw246')

In [10]:
headers=initHeaders.copy()
headers['x-requested-with']='XMLHttpRequest'
headers['x-response-format']='JSONStream'

In [11]:
loginURL = 'https://m.facebook.com/login/device-based/login/async/?refsrc=deprecated&lwv=100'
resp = session.post(loginURL, data=payloadFromHTML, headers=headers)

In [12]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='c_user', value='100087280119490', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1698821253, discard=False, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='datr', value='BcFgYz5oGxR0lki9Lt5jinXy', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1730357254, discard=False, comment=None, comment_url=None, rest={'httponly': None, 'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='fr', value='0r24tPgwuE3BGa14a.AWUZ3OD5L6OQINwSSRkiAgCOMnc.BjYMEF.5I.AAA.0.0.BjYMEF.AWVDD39Aspo', port=None, port_specified=False, domain='.facebook.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1675061250, discard=False, comment=None, com

In [13]:
resp.headers

{'Vary': 'Accept-Encoding', 'Set-Cookie': 'fr=0r24tPgwuE3BGa14a.AWUZ3OD5L6OQINwSSRkiAgCOMnc.BjYMEF.5I.AAA.0.0.BjYMEF.AWVDD39Aspo; expires=Mon, 30-Jan-2023 06:47:27 GMT; Max-Age=7775994; path=/; domain=.facebook.com; secure; httponly; SameSite=None, sb=BcFgY1TGqeaH7_pXlzry0qTd; expires=Thu, 31-Oct-2024 06:47:33 GMT; Max-Age=63072000; path=/; domain=.facebook.com; secure; httponly; SameSite=None, c_user=100087280119490; expires=Wed, 01-Nov-2023 06:47:30 GMT; Max-Age=31535997; path=/; domain=.facebook.com; secure; SameSite=None, xs=16%3Al655teTGMbetjA%3A2%3A1667285254%3A-1%3A-1; expires=Wed, 01-Nov-2023 06:47:30 GMT; Max-Age=31535997; path=/; domain=.facebook.com; secure; httponly; SameSite=None', 'report-to': '{"max_age":259200,"endpoints":[{"url":"https:\\/\\/m.facebook.com\\/ajax\\/mtouch_error_reports\\/?device_level=unknown"}]}', 'x-fb-rlafr': '0', 'content-security-policy-report-only': "default-src data: blob: 'self' https://*.fbsbx.com 'unsafe-inline' *.facebook.com 'unsafe-eval' *

In [14]:
resp.text

'for (;;);{"payload":{"actions":[{"cmd":"append","target":"static_templates","html":"","replaceifexists":true},{"cmd":"script","type":"onload","code":"(new (require(\\"ServerJS\\"))()).handle({\\"define\\":[[\\"IntlCurrentLocale\\",[],{\\"code\\":\\"ko_KR\\"},5954],[\\"CometPersistQueryParams\\",[],{\\"relative\\":{},\\"domain\\":{}},6231],[\\"CookieDomain\\",[],{\\"domain\\":\\"facebook.com\\"},6421],[\\"IntlVariationHoldout\\",[],{\\"disable_variation\\":false},6533],[\\"CurrentAdAccountInitialData\\",[],{\\"AD_ACCOUNT_ID\\":null},6828],[\\"BootloaderConfig\\",[],{\\"deferBootloads\\":false,\\"jsRetries\\":[200,500],\\"jsRetryAbortNum\\":2,\\"jsRetryAbortTime\\":5,\\"silentDups\\":false,\\"hypStep4\\":false,\\"phdOn\\":false,\\"btCutoffIndex\\":547,\\"translationRetries\\":[200,500],\\"translationRetryAbortNum\\":3,\\"translationRetryAbortTime\\":50},329],[\\"CSSLoaderConfig\\",[],{\\"timeout\\":5000,\\"modulePrefix\\":\\"BLCSS:\\",\\"loadEventSupported\\":true},619],[\\"CookieCoreCo

In [15]:
json.loads(resp.text[9:])

{'payload': {'actions': [{'cmd': 'append',
    'target': 'static_templates',
    'html': '',
    'replaceifexists': True},
   {'cmd': 'script',
    'type': 'onload',
    'code': '(new (require("ServerJS"))()).handle({"define":[["IntlCurrentLocale",[],{"code":"ko_KR"},5954],["CometPersistQueryParams",[],{"relative":{},"domain":{}},6231],["CookieDomain",[],{"domain":"facebook.com"},6421],["IntlVariationHoldout",[],{"disable_variation":false},6533],["CurrentAdAccountInitialData",[],{"AD_ACCOUNT_ID":null},6828],["BootloaderConfig",[],{"deferBootloads":false,"jsRetries":[200,500],"jsRetryAbortNum":2,"jsRetryAbortTime":5,"silentDups":false,"hypStep4":false,"phdOn":false,"btCutoffIndex":547,"translationRetries":[200,500],"translationRetryAbortNum":3,"translationRetryAbortTime":50},329],["CSSLoaderConfig",[],{"timeout":5000,"modulePrefix":"BLCSS:","loadEventSupported":true},619],["CookieCoreConfig",[],{"c_user":{"s":"None"},"cppo":{"t":86400,"s":"None"},"dpr":{"t":604800,"s":"None"},"fbl_ci":{